In [1]:
import pandas as pd
import tqdm
import numpy as np
from tqdm import tqdm
import json
import csv
import random
import ast
from base.utils import remove_unnamed_column, checknan, get_data_not_null_of_column

In [9]:
df_new_data = pd.read_csv('/home/phuongdx/python/GAPO/data/data_01-03_to_14-06_short_not_nan.csv')
df_old_data = pd.read_csv('/home/phuongdx/python/GAPO/data/data_update_processed.csv')
df_data = pd.concat([df_new_data, df_old_data])
df_data = remove_unnamed_column(df_data)
df_data.shape

(49156, 5)

In [10]:
df_data.head(10)

,post_id,image_id,content,image_src,gapo_hashtag
0,qqv8vwce7dx1,f1ff2054-b3d0-4e1e-a81e-14bcfcb98284,Thân em như tấm lụa đào\nLụa này hơi đắt \nAnh...,https://gapo-image.statics.pancake.vn/images/f...,#nu-tu
1,qqvr0i7zhqk3,f6782bfb-7327-415e-bd79-d907b611df3d,"The Remix đang sản xuất mùa tiếp theo, đ...",https://gapo-image.statics.pancake.vn/images/f...,#viet-nam
2,qqvtcmhsghi6,1a7e6c95-1f91-41e8-9d5b-269b0dbce86f,NaN,https://gapo-image.statics.pancake.vn/images/1...,#nu-tu
3,qqvteeqzjgvf,58ac3938-d32e-4fe9-9e29-4a3a71e0e89a,"""Từng yêu nhau, từng là của nhau thật lâu\nĐến...",https://gapo-image.statics.pancake.vn/images/5...,#nu-tu
4,qqvto67xm19w,6b579079-271e-47c2-b662-0d40a1f42c5c,📣THÊM 14 CA MẮC MỚI COVID-19 NHẬP CẢNH ĐÃ ĐƯỢC...,https://gapo-image.statics.pancake.vn/images/6...,#y-te
5,qqvtp35pky,57866b3b-2a7d-4ae1-90e1-40185654140d,Chẳng biết cưa chàng bắt đầu từ đâu!,https://gapo-image.statics.pancake.vn/images/5...,#nu-tu
6,qqvtschxj76i,30f5def9-67ee-48ef-98ba-64a10d8f120b,Thả thính sáng giờ mà hổng dính anh nào hết 💖💖...,https://gapo-image.statics.pancake.vn/images/3...,#nu-tu#tha-thinh
7,qqvttshp0c61,0a1214bd-3d0e-43d9-9039-e41b3c891565,"Hôm nay là cá tháng tư, yêu em thì nói, dối lò...",https://gapo-image.statics.pancake.vn/images/0...,#nu-tu
8,qqvtugl5260y,52260e5f-26e5-4994-864d-f105b04a4996,A là ai từ đâu bước đến nơi đây.\n..\n.,https://gapo-image.statics.pancake.vn/images/5...,#nu-tu
9,qqvu224irk7f,13381927-e4ea-48e0-8f94-2c49e3408b2c,Mứt dừa đi với chè xanh\nCòn e cũng phải có a ...,https://gapo-image.statics.pancake.vn/images/1...,#nu-tu


In [12]:
df_data.to_csv('/home/phuongdx/python/GAPO/data/data_final.csv')

In [13]:
def training_testing_splitter(input_reformat_path,train_path,valid_path):
    with open(input_path, 'r') as f:
        data_raw = csv.reader(f, delimiter=',', quoting=csv.QUOTE_ALL)
        next(data_raw)
        lable_dict = {}
        index_dict = {}
        for row in data_raw:
            label = row[5].split('#')[1::]
            try:
                lable_dict[label[0]] += 1
                index_dict[label[0]].append(row[0])
            except:
                lable_dict[label[0]] = 1
                index_dict[label[0]] = [row[0]]
    test_index = []
#  
    for key in lable_dict.keys():
        num_sample = int((lable_dict[key]*15)/100)
        test_index += random.sample(index_dict[key], num_sample)
        
    label_sorted = {k: v for k, v in sorted(lable_dict.items(), key=lambda item: item[1])}
#    
    count_0 = 0
    count_1 = 0
    with open(input_path, 'r') as f:
        data_raw = csv.reader(f, delimiter=',', quoting=csv.QUOTE_ALL)
        next(data_raw)
        with open(valid_path, 'w') as g:
            csv_writer = csv.writer(g, delimiter=',', quoting=csv.QUOTE_ALL)
            csv_writer.writerow(('', 'post_id', 'image_id', 'content', 'image_src', 'gapo_hashtag'))
            with open(train_path, 'w') as k:
                csv_writer_1 = csv.writer(k, delimiter=',', quoting=csv.QUOTE_ALL)
                csv_writer_1.writerow(('', 'post_id', 'image_id', 'content', 'image_src', 'gapo_hashtag'))
                for row in data_raw:

                    if row[0] in test_index:
                        csv_writer.writerow((count_0, row[1], row[2], row[3], row[4], row[5]))
                        count_0 += 1
                    else:
                        csv_writer_1.writerow((count_1, row[1], row[2], row[3], row[4], row[5]))
                        count_1 += 1
    

In [14]:
input_path = '/home/phuongdx/python/GAPO/data/data_final.csv'

train_path = '/home/phuongdx/python/GAPO/data/data_final_train.csv'
valid_path = '/home/phuongdx/python/GAPO/data/data_final_valid.csv'

train_real = '/home/phuongdx/python/GAPO/data/data_final_standard_train.csv'
valid_real = '/home/phuongdx/python/GAPO/data/data_final_standard_valid.csv' 

training_testing_splitter(input_path,train_path,valid_path)

In [18]:
def extract_labels(row):
#     labels = [i.strip() for i in row['gapo_hashtag'].split('#')[1:] if i.strip() in list_tag]
    labels = [i.strip() for i in row['gapo_hashtag'].split('#')[1:]]
    return labels

def standard_data_1(input_file, output_file):
    data = pd.read_csv(input_file)
    data['labels'] = data.apply(lambda row: extract_labels(row), axis=1)
    data = data[data.labels.str.len() != 0]
    data = data[['image_id', 'content', 'labels', 'post_id']]
    data = data.rename(columns={'content': 'content_text'})
    data.to_csv(output_file,header=False,index=False)

In [16]:
standard_data_1(train_path, train_real)
standard_data_1(valid_path, valid_real)